# Классификация Oxford Pets

Обучим сеть для классификации исходного датасета [Oxford-IIIT Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/). Загрузите датасет с Kaggle.

In [1]:
import os
from google.colab import files
import zipfile
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing.image import load_img
import seaborn as sns
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [2]:
# Установка kaggle
!pip install -q kaggle

# Создание директории для kaggle.json
!mkdir -p ~/.kaggle

# Загрузка kaggle.json
print("Загрузить kaggle.json файл.")
uploaded = files.upload()

# Перемещение kaggle.json в правильную директорию
!mv kaggle.json ~/.kaggle/

# Установка правильных разрешений
!chmod 600 ~/.kaggle/kaggle.json

# Загрузка датасета
!kaggle datasets download -d tanlikesmath/the-oxfordiiit-pet-dataset

# Распаковка датасета
with zipfile.ZipFile("the-oxfordiiit-pet-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("oxford-iiit-pet")

print("Датасет успешно загружен и распакован в директорию 'oxford-iiit-pet'")

# Вывод списка файлов в распакованной директории
print("\nСодержимое директории 'oxford-iiit-pet':")
!ls oxford-iiit-pet

Загрузить kaggle.json файл.


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/tanlikesmath/the-oxfordiiit-pet-dataset
License(s): CC-BY-SA-4.0
 99% 1.46G/1.48G [00:20<00:00, 125MB/s]
100% 1.48G/1.48G [00:20<00:00, 76.6MB/s]
Датасет успешно загружен и распакован в директорию 'oxford-iiit-pet'

Содержимое директории 'oxford-iiit-pet':
images


Для визуализации изображений используйте функцию:

In [3]:
def display_images(l,titles=None,fontsize=12):
    n=len(l)
    fig,ax = plt.subplots(1,n)
    for i,im in enumerate(l):
        ax[i].imshow(im)
        ax[i].axis('off')
        if titles is not None:
            ax[i].set_title(titles[i],fontsize=fontsize)
    fig.set_size_inches(fig.get_size_inches()*n)
    plt.tight_layout()
    plt.show()

In [4]:
len(os.listdir('oxford-iiit-pet/images'))

7394

In [5]:
# Путь к папке с изображениями
source_folder = 'oxford-iiit-pet/images'
# Папка, в которую будут перемещены изображения по классам
destination_folder = 'oxford-iiit-pet/organized_images'

# Создаем папку для организованных изображений, если она не существует
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Перебираем все файлы в исходной папке
for filename in os.listdir(source_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Определяем класс по имени файла
        class_name = filename.split('_')[0]
        class_folder = os.path.join(destination_folder, class_name)

        # Создаем директорию для класса, если она не существует
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)

        # Перемещаем файл в папку соответствующего класса
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(class_folder, filename)
        shutil.move(source_path, destination_path)

print("Изображения успешно организованы по классам.")

Изображения успешно организованы по классам.


In [6]:
path_to_dataset = 'oxford-iiit-pet/organized_images'

In [14]:
# Создание генераторов для train, val и test
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(path_to_dataset,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    subset='training')

val_generator = train_datagen.flow_from_directory(path_to_dataset,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical',
                                                  subset='validation')

test_generator = test_datagen.flow_from_directory(path_to_dataset,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 5913 images belonging to 35 classes.
Found 1477 images belonging to 35 classes.
Found 7390 images belonging to 35 classes.


In [8]:
# Загрузка предобученной сети VGG-16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Замораживание весов предобученной сети
base_model.trainable = False

# Создание классификатора
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator)





58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


185/185 ━━━━━━━━━━━━━━━━━━━━ 135s 632ms/step - accuracy: 0.0674 - loss: 3.7942 - val_accuracy: 0.1469 - val_loss: 2.9696
Epoch 2/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 111s 541ms/step - accuracy: 0.2210 - loss: 2.6668 - val_accuracy: 0.2722 - val_loss: 2.3720
Epoch 3/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 103s 543ms/step - accuracy: 0.3288 - loss: 2.1461 - val_accuracy: 0.3399 - val_loss: 2.1156
Epoch 4/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 104s 548ms/step - accuracy: 0.4073 - loss: 1.8734 - val_accuracy: 0.3521 - val_loss: 2.0235
Epoch 5/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 104s 548ms/step - accuracy: 0.4507 - loss: 1.7137 - val_accuracy: 0.4103 - val_loss: 1.9146
Epoch 6/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 142s 546ms/step - accuracy: 0.4707 - loss: 1.6098 - val_accuracy: 0.4265 - val_loss: 1.8531
Epoch 7/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 141s 538ms/step - accuracy: 0.5060 - loss: 1.5192 - val_accuracy: 0.4177 - val_loss: 1.8675
Epoch 8/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 142s 542ms/step - accuracy: 0.5512 - loss: 1.39

In [9]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'VGG-16 Test accuracy: {test_acc:.2f}')

231/231 ━━━━━━━━━━━━━━━━━━━━ 46s 199ms/step - accuracy: 0.6002 - loss: 1.2282
VGG-16 Test accuracy: 0.60


In [10]:
# Загрузка предобученной сети VGG-19
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Замораживание весов предобученной сети
base_model.trainable = False

# Создание классификатора
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(len(train_generator.class_indices), activation='softmax')(x)

model2 = Model(inputs=base_model.input, outputs=x)

# Компиляция модели
model2.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
history = model2.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator)


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 111s 573ms/step - accuracy: 0.0736 - loss: 3.7757 - val_accuracy: 0.1686 - val_loss: 2.9885
Epoch 2/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 106s 554ms/step - accuracy: 0.2084 - loss: 2.7536 - val_accuracy: 0.2742 - val_loss: 2.4008
Epoch 3/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 142s 558ms/step - accuracy: 0.3162 - loss: 2.1844 - val_accuracy: 0.3541 - val_loss: 2.1318
Epoch 4/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 142s 562ms/step - accuracy: 0.4105 - loss: 1.8865 - val_accuracy: 0.3602 - val_loss: 2.0272
Epoch 5/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 105s 554ms/step - accuracy: 0.4708 - loss: 1.6794 - val_accuracy: 0.3988 - val_loss: 1.9622
Epoch 6/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 104s 552ms/step - accuracy: 0.5064 - loss: 1.5831 - val_accuracy: 0.4252 - val_loss: 1.8298
Epoch 7/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 105s 555ms/step - accuracy: 0.5514 - loss: 1.4399 - val_accuracy: 0.3981 - val_loss: 1.9452
Epoch 8/10
185/185 ━━━━━━

In [11]:
test_loss, test_acc = model2.evaluate(test_generator)
print(f'VGG-19 Test accuracy: {test_acc:.2f}')

231/231 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - accuracy: 0.6569 - loss: 1.0751
VGG-19 Test accuracy: 0.66


In [ ]:
# Загрузка предобученной сети ResNet-50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Замораживание весов предобученной сети
base_model.trainable = False

# Создание классификатора
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(len(train_generator.class_indices), activation='softmax')(x)

model3 = Model(inputs=base_model.input, outputs=x)

# Компиляция модели
model3.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
history = model3.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator)



Epoch 1/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 120s 582ms/step - accuracy: 0.0385 - loss: 3.6220 - val_accuracy: 0.0542 - val_loss: 3.5350
Epoch 2/10
120/185 ━━━━━━━━━━━━━━━━━━━━ 29s 447ms/step - accuracy: 0.0611 - loss: 3.5290

In [13]:
test_loss, test_acc = model3.evaluate(test_generator)
print(f'ResNet Test accuracy: {test_acc:.2f}')



231/231 ━━━━━━━━━━━━━━━━━━━━ 37s 159ms/step - accuracy: 0.0814 - loss: 3.3212
ResNet Test accuracy: 0.08


Лучшие результаты test_accuracy показаны на сети с предобученной VGG19. Сеть с ResNet почти не обучается и показывает самые низкие результаты, изменение парамаетров не помогло